In [22]:
#grab knapsack data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
cp -r ./drive/MyDrive/knapsack-data/*.txt ./

In [4]:
"""
Use a genetic algorithm to 
solve the knapsack problem
"""
import numpy as np
from matplotlib import pyplot as plt
from numpy.random import randn, rand
from random import randint
import operator
from copy import copy
from PIL import Image
import os
import pandas as pd

# TODO: implement crossover 

In [71]:
def read_file(filename):
    """
    Given a filename with knapsack data,
    read number,capacity,values,weights and return.
    """
    fname=filename[0:len(filename)-4].split('_')
    number=fname[0]
    capacity=fname[1]
    df = pd.read_csv(filename, delimiter=" ", names=["Value","Weight"])
    values = df['Value'].to_numpy()
    weights = df['Weight'].to_numpy()
    #drop first in values and weights column as it is number, capacity
    return int(number),int(capacity),values[1:],weights[1:]

def init_population(n,l):
    """
    initialize n candidates
    of length l
    """
    population=[list(np.random.choice([0, 1], size=(l,))) for i in range(n)]
    return population

def mutate(population, mut_rate):
    """
    Randomly mutate the candidates.
    Mutation is done by picking candidates 
    and flipping a single bit
    """
    population = np.array(population)
    mut_population=[]
    mut_count = 0
    #pick some candidates to randomly mutate
    cands = np.random.choice(range(len(population)),replace=False, size=(mut_rate,))
    #print("Mutating cands ",cands)
    for i in cands:
        #print(f"Mutating candidate {population[i]}")
        j = np.random.choice(range(len(candidate)),replace=False, size=(1,))
        #print(f"Mutating index {j}")
        if population[i][j] == 0:
            population[i][j] = 1
        else:
            population[i][j] = 0
        #print(f"Mutated candidate {population[i]}")
    return list(population)

def selection(n_remove,cap,values,weights,population):
    """
    Truncation selection.
    Given a populationulation specified by population
    remove the n individuals with lowest fitness.
    Invalid combinations (exceeds weight of knapsack)
    are set to fitness 0.
    """
    score = {}
    for i,candidate in enumerate(population):
        fit = fitness(cap,values,weights,candidate)
        score[f"{i}"] = fit

    #get rid of weakest individuals
    sorted_score = dict(sorted(
            score.items(),
            key=operator.itemgetter(1),
            ))

    #indexes to be eliminated
    perished_inds = list(sorted_score.keys())[0:n_remove]
    perished_inds=[int(i) for i in perished_inds]

    #remove from population perished individuals
    for i in sorted(perished_inds, reverse=True):
        del population[i]
    
    return population

def fitness(cap,values,weights,candidate):
    """
    Evaluate the fitness of a candidate
    i.e if we have 
    cap=3
    w=[1,2,3]
    v=[10,10,10]
    bs1 = [1,1,0] has f1=20 (w1=1+2=3, < cap)
    bs2 = [1,1,1] has f2=0 (w2=1+2+3=6 > cap so invalid)
    """
    total_val = np.dot(candidate, values)
    total_weight = np.dot(candidate, weights)
    if total_weight > cap:
        return 0
    else:
        return total_val

In [78]:
bs_len,cap,values,weights = read_file("10_269.txt")
print(f"Number of items: {bs_len}\n Capacity: {cap}")
print(f"Object values: {values}\n Object weights: {weights}")

pop_num = 1000
#initialise pop_num individuals of length bs_len
population = init_population(pop_num, bs_len)
print("pop init: ",population)
for i in range(99):
    fits = [fitness(cap,values,weights,candidate) for candidate in population]
    print("Mean fitness: ",np.mean(fits))
    population = selection(10,cap,values,weights,population)
    population = mutate(population,10)
print(population)
print(fits)

Number of items: 10
 Capacity: 269
Object values: [55 10 47  5  4 50  8 61 85 87]
 Object weights: [95  4 60 32 23 72 80 62 65 46]
pop init:  [[0, 1, 1, 1, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 1, 1], [1, 1, 1, 1, 0, 0, 0, 1, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 1], [0, 1, 1, 0, 1, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 0, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1, 1, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 1, 1], [0, 1, 0, 0, 1, 0, 0, 1, 0, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 0, 0, 0, 1, 0, 0], [1, 1, 1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 0, 0, 1, 0, 1, 1, 0], [1, 0, 1, 1, 0, 1, 1, 1, 0, 1], [1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 1, 1, 0, 1, 1, 